In [1]:
!nvidia-smi

Sun Jun  4 02:07:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.51.01              Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:09:00.0  On |                  N/A |
|  0%   46C    P8               31W / 350W|   1153MiB / 24576MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from PIL import Image
import pandas as pd
import os

from scripts import Worker

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [23]:
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)

class Net(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(1, 32, 3, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1), nn.MaxPool2d(2), nn.Dropout2d(0.25),
            Flatten(), nn.Linear(9216, 128), nn.ReLU(), nn.Dropout2d(0.5),
            nn.Linear(128, 2), nn.LogSoftmax(dim=1) )

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx/len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss/len(test_loader.dataset), correct, len(test_loader.dataset),
        100. * correct/len(test_loader.dataset)))

In [19]:
class MyDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.data_dir = data_dir
        self.data_info = self.get_img_info(data_dir)
        self.transforms = transforms

    def __getitem__(self, item):
        path_img, label = self.data_info.iloc[item][1:3]
        label = int(label)
        path_img = os.path.join(self.data_dir, path_img)
        image = Image.open(path_img).convert('RGB') # Gray scale is enough for logic interpretation
        # 使用定义好的transforms，对数据进行处理
        if self.transforms is not None:
            image = self.transforms(image)

        return image, label

    def __len__(self):
        return len(self.data_info)
    
    def get_img_info(self, data_dir):
        path_dir = os.path.join(data_dir, 'label.csv')
        return pd.read_csv(path_dir)

In [20]:
dataset_folder = "dataset03"
exp_name = f"{dataset_folder}_ablation_study"


In [21]:
kwargs = {'num_workers': 16}
batch_size=1

InDL = True
if InDL:
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

    trainset = MyDataset(f"data/train/{dataset_folder}", transform)
    testset = MyDataset(f"data/test/{dataset_folder}", transform)
else:
    transform=transforms.Compose(
        [transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

    trainset = datasets.MNIST(root='data/mnist', train=True, download=True, transform=transform)
    testset = datasets.MNIST(root='data/mnist', train=False, download=True, transform=transform)
    
    # trainset = datasets.CIFAR10(root='data/cifar10', train=True, download=True, transform=transform)
    # testset = datasets.CIFAR10(root='data/cifar10', train=False, download=True, transform=transform)
    

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, **kwargs)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, **kwargs)

In [22]:
epochs, lr = 20, 1e-3
model_name = "1-net"

if InDL:
    model_name = f"{model_name}-InDL"
else:
    model_name = f"{model_name}-MNIST"

model = Net().to(device)

worker = Worker(exp_name, model_name, model, trainloader, testloader, epochs=epochs, lr=lr)
worker.train()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x246016 and 9216x128)

In [ ]:
from sklearn.metrics import classification_report

state_dict = torch.load(f"models/{exp_name}/{model_name}.pt")
model = Net().to(device)
model.load_state_dict(state_dict)

def evaluate_model(model, dataloader, device):
    model.eval()  # set the model to evaluation mode
    
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():  # don't calculate gradients
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    print(classification_report(all_labels, all_predictions))

# Assuming you have a model, dataloader and device (CPU or CUDA) defined
evaluate_model(model, testloader, device)

FileNotFoundError: [Errno 2] No such file or directory: 'models/dataset03_ablation_study/1-net-InDL.pt'